# Importing Required Libs

In [ ]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42

In [ ]:
import seaborn as sns
import matplotlib.image as immg
import gc
import numpy as np
import random
from PIL import Image
import cv2
import imageio

# 1) Train Data Read

In [ ]:
train = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
train
#train csv imageid-celltype-pixeldetails in which cell is present

In [ ]:
train['file_path'] =train['id'].apply(lambda x: '../input/sartorius-cell-instance-segmentation/train/{}.png'.format(x))
train.head()
#in train csv , we have mapped the image id with image location and stored it in the df col file path

In [ ]:
#unique Images
uid = train.id.unique()
print("no.of unique images")
len(uid)

### Functions for generating Masks using pixel information provided in train.csv

In [ ]:
#Masking functions
def get_image(path):
    image = np.array(Image.open(path))
    #image=cv2.imread(Image.open(path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def get_annot(img_id):
    #train[train["id"] == id_]["annotation"].tolist()
    return train[train.id == img_id]["annotation"].tolist()

def get_mask(img_annotations,colours=True): 
    if colours:
        mask = np.zeros((520, 704, 3))
        for annot in img_annotations:
            mask += rle_decode(annot, shape=(520, 704, 3), color=np.random.rand(3))
    else:
        mask = np.zeros((520, 704, 1))
        for annot in img_annotations:
            mask += rle_decode(annot, shape=(520, 704, 1),color = 1)
    mask = mask.clip(0, 1)
    return mask



In [ ]:
def rle_decode(mask_rle, shape, color=1):

    s = mask_rle.split()
    
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
            
    for start, end in zip(starts, ends):
        img[start : end] = color
    
    return img.reshape(shape)

### Understanding masking functions and visualizations

In [ ]:
#OriginalImage
im=np.array(Image.open('../input/sartorius-cell-instance-segmentation/train/0030fd0e6378.png'))
plt.title("Original Image")
plt.imshow(im)

In [ ]:
#Image after BGR2RGB conversion
im2=get_image('../input/sartorius-cell-instance-segmentation/train/0030fd0e6378.png')
plt.title("Image after BGR2RGB conversion")
plt.imshow(im2)

In [ ]:
print(uid[0])

In [ ]:
ann=get_annot(uid[0])# will get the pixel(annotation) values where the cell is precent of all cells present in a image w.r.t imgID
print(len(ann))
im_mask=get_mask(ann,colours=False)#will generate a mask with values from annotations
plt.imshow(im_mask)

In [ ]:
#Visualizing images and masks

plt.figure(figsize=(20, 20))

plt.subplot(1, 3, 1)
plt.imshow(im)
plt.title('Original image')

plt.subplot( 1, 3, 2)
plt.imshow(im_mask)
plt.title('Mask')

plt.subplot( 1, 3, 3)
plt.imshow(im)
plt.imshow(im_mask,alpha=0.2)
plt.title('Both')
plt.tight_layout()
plt.show()

# 2. Preparing the data

In [ ]:
# Set some parameters
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1
X_TRAIN_PATH = '../input/sartorius-cell-instance-segmentation/train/'
X_TEST_PATH = '../input/sartorius-cell-instance-segmentation/test/'

In [ ]:
# Get trainIDs
train_ids = train['id'].unique().tolist()
print(len(train_ids))
print(train_ids[0])

In [ ]:
# Get and resize train images and masks
X_train = np.zeros((train['id'].nunique(), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((train['id'].nunique(), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

In [ ]:
print('Getting and resizing train images')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = X_TRAIN_PATH + id_
    img = imread(path + '.png')[:,:]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    img = np.expand_dims(img, axis = 2)
    X_train[n] = img
    mask = np.zeros((520, 704, 1))
    annots = get_annot(id_)
    mask=get_mask(annots,colours=False)
    mask = mask[:,:,0]
    mask = np.expand_dims(resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
    
    Y_train[n] = mask
    
print("done")

In [ ]:
print(len(X_train))
print(X_train.shape)

In [ ]:
print(len(Y_train))
print(Y_train.shape)

In [ ]:
#Getting Test Ids
test_ids = next(os.walk(X_TEST_PATH))[2]
print(test_ids)

In [ ]:
# Get and resize test images

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
print('Getting and resizing test images ... ')
sys.stdout.flush()

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    print(n, id_)
    path = '../input/sartorius-cell-instance-segmentation/test/'+id_
    img = imread(path)[:,:]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    img = np.expand_dims(img, axis = 2)
    X_test[n] = img

print('Done!')

In [ ]:
sample_id_num = 8
plt.imshow(X_train[sample_id_num][:,:,0])
plt.show()
plt.imshow(Y_train[sample_id_num][:,:,0])
plt.show()

# 3)Metric Definitions

In [ ]:
tf.to_int32=lambda x: tf.cast(x, tf.int32)

# Define IoU metric
def mean_iou(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return tf.convert_to_tensor(iou)

In [ ]:
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

# 4) Unet

In [ ]:
# Building a U-Net model
#functional
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
la = Lambda(lambda x: x / 255) (inputs)

ec1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(la)#(None, 256, 256, 16)
ec1 = Dropout(0.1) (ec1)#layer Dropout#shape=(None, 256, 256, 16)
ec_f1 = Conv2D(16,(3,3),activation='relu', kernel_initializer='he_normal', padding='same') (ec1)
ec_f1_red = MaxPooling2D((2, 2)) (ec_f1)#reduced shape=(None, 128, 128, 16)

ec2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(ec_f1_red)#shape=(None, 128, 128, 32)
ec2 = Dropout(0.1) (ec2)#layer Dropout#shape=(None, 128, 128, 132)
ec_f2 = Conv2D(32,(3,3),activation='relu', kernel_initializer='he_normal', padding='same') (ec2)#shape=(None, 128, 128, 32)
ec_f2_red = MaxPooling2D((2, 2)) (ec_f2)#reduced shape=(None, 64, 64, 32)

ec3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (ec_f2_red)#shape=(None, 64, 64, 64)
ec3 = Dropout(0.2) (ec3)#layer Dropout#shape=(None, 64, 64, 64)
ec_f3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (ec3)#shape=(None, 64, 64, 64)
ec_f3_red = MaxPooling2D((2, 2)) (ec_f3)#reduced shape=(None, 32, 32, 64)

ec4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (ec_f3_red)#shape=(None, 32, 32, 128)
ec4 = Dropout(0.2) (ec4)#shape=(None, 32, 32, 128)
ec_f4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (ec4)#shape=(None, 32, 32, 128)
ec_f4_red = MaxPooling2D(pool_size=(2, 2)) (ec_f4)#shape=(None, 16, 16, 128)

ec5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (ec_f4_red)#shape=(None, 16, 16, 256)
ec5 = Dropout(0.3) (ec5)#shape=(None, 16, 16, 256)
ec_f5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (ec5)#shape=(None, 16, 16, 256)

duc1 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (ec_f5)
dconc1 = concatenate([duc1, ec_f4])
dc1 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (dconc1)
dc1 = Dropout(0.2) (dc1)
dc_f1 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (dc1)

duc2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (dc_f1)
dconc2 = concatenate([duc2 , ec_f3])
dc2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (dconc2)
dc2 = Dropout(0.2) (dc2)
dc_f2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (dc2)

duc3 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (dc_f2)
dconc3 = concatenate([duc3, ec_f2])
dc3 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (dconc3)
dc3 = Dropout(0.1) (dc3)
dc_f3 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (dc3)

duc4 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (dc_f3)
dconc4 = concatenate([duc4, ec_f1], axis=3)
dc4 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (dconc4)
dc4 = Dropout(0.1) (dc4)
dc_f4 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (dc4)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (dc_f4)
model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coefficient,mean_iou])
model.summary()


In [ ]:
# Fit model
earlystopper = EarlyStopping(patience=16, verbose=1)
checkpointer = ModelCheckpoint('best_model.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.15, batch_size=5, epochs=100,callbacks=[earlystopper, checkpointer])

In [ ]:
plt.plot(results.history['dice_coefficient'])
plt.plot(results.history['val_dice_coefficient'])
plt.title('dice_coefficient')
plt.ylabel('dice_coefficient')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
plt.plot(results.history['mean_iou'])
plt.plot(results.history['val_mean_iou'])
plt.title('mean_iou')
plt.ylabel('mean_iou')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

# 5)Predictions

In [ ]:
# Predictions
#model = load_model('best_model.h5', custom_objects={'dice_coefficient': dice_coefficient})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Applying Threshols
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)
'''
# Threshold predictions
preds_train_t = (preds_train > 0.6).astype(np.uint8)
preds_val_t = (preds_val > 0.6).astype(np.uint8)
preds_test_t = (preds_test > 0.6).astype(np.uint8)
# Threshold predictions
preds_train_t = (preds_train > 0.7).astype(np.uint8)
preds_val_t = (preds_val > 0.7).astype(np.uint8)
preds_test_t = (preds_test > 0.7).astype(np.uint8)'''

# list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True))

In [ ]:
# sanity check on a random training sample
ix = 25
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(Y_train[ix]))
plt.show()
imshow(np.squeeze(preds_train_t[ix]))
plt.show()

In [ ]:
# Performing a sanity check on a random validation sample
ix = 6
imshow(X_train[int(X_train.shape[0]*0.9):][ix])
plt.show()
imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()

In [ ]:
# Test samples
for ix in range(len(X_test)): 
    print(ix)
    plt.title('Image')
    imshow(X_test[ix])
    plt.show()
    plt.title('Predicted Mask')
    imshow(np.squeeze(preds_test_t[ix]))
    plt.show()

In [ ]:
#Visualizing images and masks
for ix in range(len(X_test)):
    
    print('for image',ix+1)
    im=X_test[ix]
    im_mask=np.squeeze(preds_test_t[ix])

    plt.figure(figsize=(20, 20))

    plt.subplot(1, 3, 1)
    plt.imshow(im[:,:,0])
    plt.title('Test image')

    plt.subplot( 1, 3, 2)
    plt.imshow(im_mask)
    plt.title('Predicted Mask')

    plt.subplot( 1, 3, 3)
    plt.imshow(im)
    plt.imshow(im_mask,alpha=0.2)
    plt.title('Both')
    plt.tight_layout()
    plt.show()

# 6) Resizing

In [ ]:
sub=pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')
sub.head()

In [ ]:
pred =preds_test_t[0]
plt.imshow(pred)
plt.title('pred before resize')
plt.axis("off")
plt.show()

In [ ]:
#test_mask: after reshape 
test_masks=[]
for pred in preds_test_t:
    print(pred.shape)
    t_mask=cv2.resize(pred,dsize=(704,520),interpolation=cv2.INTER_CUBIC).reshape(520,704,1)
    test_masks.append(t_mask)
    print(t_mask.shape)  
    plt.imshow(t_mask)
    plt.title('pred after resize')
    plt.axis("off")
    plt.show()
len(test_masks)

# 7)Fixoverlap

Reference: https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/279995

In [ ]:
#rle_encoding
def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))

In [ ]:
def check_overlap(msk):
    msk = msk.astype(np.bool).astype(np.uint8)
    return np.any(np.sum(msk, axis=-1)>1)

In [ ]:
def fix_overlap(msk):
    """
    Args:
        mask: multi-channel mask, each channel is an instance of cell, shape:(520,704,None)
    Returns:
        multi-channel mask with non-overlapping values, shape:(520,704,None)
    """
    msk = np.array(msk)
    msk = np.pad(msk, [[0,0],[0,0],[1,0]])
    ins_len = msk.shape[-1]
    msk = np.argmax(msk,axis=-1)
    msk = tf.keras.utils.to_categorical(msk, num_classes=ins_len)
    msk = msk[...,1:]
    msk = msk[...,np.any(msk, axis=(0,1))]
    return msk

In [ ]:
for test_mask in test_masks:
    overlap_test_masks=check_overlap(test_mask)
    print(overlap_test_masks)

In [ ]:
#test_mask2: after reshape after fix_overlapping
test_masks2=[]
for test_mask in test_masks:
    test_mask2 = fix_overlap(test_mask).reshape(520,704,1)
    print(test_mask2.shape)
    test_masks2+=[test_mask2]

In [ ]:
for test_mask2 in test_masks2:
    overlap_test_masks2=check_overlap(test_mask2)
    print(overlap_test_masks2)

In [ ]:
predicted2 = [rle_encoding(test_mask2) for test_mask2 in test_masks2]
#print(predicted2[0])

# 8)Writng to csv

In [ ]:
submit = sub.copy()
submit['predicted'] = predicted2
submit.to_csv('submission.csv', index=False)
submit

ReferencesUsed:
* https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/293159
* https://www.kaggle.com/karan23258/cell-instance-segmentation-unetfromscratch